## Donkey2.1
This is a notebook to explore a change in Donkey archetecture to make it more modular. 

### Problem: 
Currently Donkey is written to use only images, steering angles, and throttle values. Changing anything about the code requires rewriting many different parts.

### Solution:
Standardize the way data is passed between, sensors, actuators, pilots and controllers so that a vehicle can more easily manipulated. Specifically the change should:
1. Enable contributors to create, test and use small code moduals withou having to rewrite unrelated  code sections. 
2. Support an arbitrary number of inputs(camera, lidar, ...) and outputs(actuators, ...). 
3. Make the car creation script human readable. 


### Architecture
These goals could be achieved by separating the concerns each majore aspects of the vehicle. Here are the functions of in this architecture: 
1. Vehicle - a container class to hold and manage all aspects of the vehicle. 
2. Parts - modular components of the vehicle that read/write to the memory. This includes sensors, actuators, remote controlers and a datastore. 
3. Memory - holds the state of the vehicle and is used to pass variables between parts. 
4. Drive loop - a function of the vehicle that runs ensures each part interacts with the memory.


![arch](images/donkey21_arch.png)

### Example
Here is an example of how to setup a car that only has a person that controlls it. 

```python

#setup the parts
camera = Camera(resolution=(120,160), refresh_rate=.1, threaded=True)
controller = LocalWebController()
throttle = ThrottleActuator(pwm_max=500, pwm_min=200, channel=0)
steering = SteeringActuator(pwm_left=400, pwm_center=500, pwm_right=600, channel=1)
datastore = FileDatastore(path='~/mydonkey/sessions/')

#create the vehicle
car = Vehicle()

#add a camera and remote controller
car.add(camera, outputs=['camera/image'], threaded=True)
car.add(controller, inputs=['camera/image'], outputs=['user/throttle', 'user/angle', 'user/drivemode'])

#get the final throttle and steering values to move the vehicle.
car.add(throttle, inputs=['user/throttle'])
car.add(steering, inputs=['user/steering'])

#record all the variables in memory every looop.
car.add(datastore, inputs='*')

#start the drive loop
car.start()
```

## Parts 
The majority of the vehicle code will be in parts. Parts are python classes that have a common structure that can be loaded by the vehicle and run in the drive loop. Processes that take a long time to run can be threaded so that they do not block the drive loop. Here is an example part that can be run threaded or not. 

In [5]:
import time, random 

class RandPart():
    name = "Part Base class"
    
    def __init__(self):
        self.num = 0.
    
    def update(self):
        #Threaded function.
        self.running=True
        while self.running:
            self.num = self.run()
    
    def run(self):
        #Called in drive loop if not threaded.
        time.sleep(.5)
        return [random.randint(0, 100)]
    
    def run_threaded(self):
        #What's called in drive loop if part is threaded.
        return self.num

In [7]:
part = RandPart()
part.run()

[56]

We can see that the part takes .5 seconds to process. When this part is in the vehicle, we don't want this part to hold up the execution of other processes so we can make it threaded like this.  

In [9]:
from threading import Thread

t = Thread(target=part.update, args=())
t.daemon = True
t.start()

for _ in range(10):
    print(part.run_threaded())
    time.sleep(.1)

part.running=False
t.join()

[32]
[32]
[32]
[32]
[32]
[88]
[88]
[88]
[88]
[88]


### Memory 
The memory's sole job is to hold and transmit data to the parts of the vehicle. This includes outputs of sensors, inputs to actuators and all the connecting state variables required by autopilots and remote controlls. 

While it is not intended to persist the values over time it could be designed to save an arbitrary number of iterations of the drive loop in a ring queue or something that had quick access. 

Longterm storage of the data stored in memory could implemented with a Datastore part that could save the data to the file system. 

In [11]:
class Memory():
    def __init__(self):
        self.dict = {}
        pass
    
    def put(self, keys, inputs):
        for i, key in enumerate(keys):
            self.dict[key] = inputs[i]
            
    def get(self, keys):
        result = [self.dict.get(k) for k in keys]
        return result
        
m = Memory()
m.put(['name'], ['donkey'])
m.get(['name'])

['donkey']

### Vehicle

In [61]:
import random
import time

class Vehicle():
    def __init__(self, mem=None):
        
        if not mem:
            mem = Memory()
        self.mem = mem
        
        self.parts = [] 
        self.on = True
        threads = []
        
    def add(self, part, inputs=[], outputs=[], threaded=False):
        """ 
        Method to add a part.
        
        inputs: list of variable names to get from memory
        ouputs: list of variable names to save to memory
        threaded: boolean indicating if part should be run in separate thread
        """
        
        p = part
        print('Adding part {}.'.format(p.name))
        entry={}
        entry['part'] = p
        entry['inputs'] = inputs
        entry['outputs'] = outputs
        
        if threaded:
            t = Thread(target=part.update, args=())
            t.daemon = True
            entry['thread'] = t
            
        self.parts.append(entry)
    
    
    def start(self, delay=.1):
        """ 
        Start the threaded parts and the drive loop. 
        
        delay: seconds to sleep after each drive loop
        
        """
        
        
        for entry in self.parts:
            if entry.get('thread'):
                #start the update thread
                entry.get('thread').start()
        
        #wait until the sensors/actuators warm up.
        print('Starting vehicle...')
        time.sleep(1)
        
        count = 0
        while self.on:
            count += 1
            
            for entry in self.parts:
                p = entry['part']
                #get inputs from memory
                inputs = self.mem.get(entry['inputs'])
                
                #run the part
                if entry.get('thread'):
                    outputs = p.run_threaded(*inputs)
                else:
                    outputs = p.run(*inputs)
                
                #save the output to memory
                self.mem.put(entry['outputs'], outputs)
                
                time.sleep(delay)
                
            #for testing stop the car after 10 iterations
            if count > 10: self.on = False



In [62]:
class PrintPart():
    """ A part to print variables we select from memory. """
    
    name = "PrintPart"
    
    def run(self, *args):
        #Called in drive loop if not threaded.
        print('PrintPart printing: ', end = ' ')
        print(*args)

In [63]:
V = Vehicle()

In [64]:
V.add(RandPart(), outputs=['rand/num'], threaded=True)
V.add(PrintPart(), inputs=['rand/num'])

Adding part Part Base class.
Adding part PrintPart.


In [65]:
V.start()


Starting vehicle...
PrintPart printing:  4
PrintPart printing:  4
PrintPart printing:  4
PrintPart printing:  59
PrintPart printing:  59
PrintPart printing:  19
PrintPart printing:  19
PrintPart printing:  19
PrintPart printing:  24
PrintPart printing:  24
PrintPart printing:  51
